In [1]:
import sys


import PredictSeq as ps
import time
import numpy as np
import util.AA_Exchange as aa
from pymol import cmd, stored, selector
from sklearn.metrics import confusion_matrix
import pandas as pd
import pickle
import os
import RelaxFastdesign as ros
import pickle

import argparse
import seaborn as sns
import time
import pyrosetta
import matplotlib.pyplot as plt
pyrosetta.init("-beta -mute all")


ModuleNotFoundError: No module named 'pyrosetta'

In [2]:
import Test_GraphGen as tgg
#-graph gen test statistics for 

In [17]:
gg_net = 'data/best_checkpoint_epoch67_graphGenData.pt'
net = 'data/Best_GraphGen.pt'
modelDirec = '../data/4H_dataset/models/'
straightDirec = '../data/4H_dataset/str_models/'

limit=100000

In [21]:
sA[0]

'SLDEILETLKELGVPPELLEEVKKVLEELERRGDPKREEVRRLLEEARKVPDPREVRRLVEEAVR'

In [22]:
sP[0]

['SLEELLEEARKLGVPPELLERVEELLKELEERGDPRREEVRRLLEEIVKIPDPEEARKLLEEAIK',
 '0002182ba9e87d7b0d6b7d16b7001923_0001']

In [18]:
start_pred = time.time()
        
#returns a sequence/ name of the protein
sP = ps.getSeq(net, direcName = modelDirec,dtest=straightDirec, 
       limit = limit, hidden=64, mpnn=True,test=True, train=False,catSet=False)
end_pred = time.time()

sA = tgg.get_pdb_seq(sP,direcName=modelDirec,removeEnding=False)
print('Statistics for straight helix trained GraphGen')
matrix = tgg.compare_true_predicted(sP,sA)
tgg.group_accuracy(matrix)  #prints accuracy



size of directory straight is 27877
size of sequence directory is 27894
Statistics for straight helix trained GraphGen
GroupCharge 95%
GroupPhobic 96%
Hydrophobic Accuracy AILVF 70%
Charged Accuracy DE_RK 62%
 AILVF [74 27 43 81 59]
 DE KR [75 75 44 48]
 P G [93 96]



In [19]:
sG = ps.getSeq(gg_net, direcName = modelDirec,dtest=straightDirec, 
               limit = limit, hidden=128, mpnn=True,test=True, train=False,catSet=False)
               
print('Statistics for originally trained GraphGen Network')
matrix = tgg.compare_true_predicted(sG,sA)
tgg.group_accuracy(matrix)  #prints accuracy

size of directory straight is 27877
size of sequence directory is 27894
Statistics for originally trained GraphGen Network
GroupCharge 58%
GroupPhobic 81%
Hydrophobic Accuracy AILVF 30%
Charged Accuracy DE_RK 27%
 AILVF [75  9  9 35  8]
 DE KR [34 38 18  9]
 P G [15  4]



In [25]:
#----------------predict_relax------------
modelDirec = '../data/4H_dataset/models/'
straightDirec = '../data/4H_dataset/str_models/'

In [26]:
outDirec = '../Figures/GraphGen_Figure/output/gg_orig_test'
scoreDirec = '../Figures/GraphGen_Figure/output/'
outName = 'gg_orig'

In [44]:
def relax_stats(seqList,outName,outDirec,scoreDirec, dOrig= modelDirec, dStraight = straightDirec, scoreList=[],nameList=[]):
    
    labels = ['name','pred_seq','pred_scores','pred_time','orig_scores']

    
    
    for x in range(len(seqList)):
        seq = seqList[x][0]
        name = seqList[x][1]

        #change residues to graphGen prediction
        start = time.time()
        pose = pyrosetta.pose_from_file(f'{dStraight}/{name}_str.pdb')
        pred_pose = ros.mutate_residue(pose,seq)
        pose_relax = ros.fastRelax(pred_pose)
        end = time.time()
        
        pose_relax.dump_pdb(f'{outDirec}/{name}_{outName}.pdb')

        scoreList.append([name, seq, ros.get_pose_scores(pred_pose), end-start])
        nameList.append(name)
        
    tgg.save_scoreList(scoreDirec,nameList=nameList,scoreList=scoreList)
    
    return nameList, scoreList

In [1]:
# sG = ps.getSeq(gg_net, direcName = modelDirec,dtest=straightDirec, 
#                limit = 2, hidden=128, mpnn=True,test=True, train=False,catSet=False)
# nL, sL = relax_stats(sG,outName,outDirec,scoreDirec)
    

In [6]:
def openArray(name,direc='../Figures/GraphGen_Figure/output/'):
    
    with open(f'{direc}/{name}_nameList.pkl', 'rb') as f:
        nameList = pickle.load(f)
    with open(f'{direc}/{name}_scoreList.pkl', 'rb') as f:
        scoreList = pickle.load(f)
        
    return nameList, scoreList
        


In [ ]:
#labels = ['name','pred_seq','pred_scores','pred_time']

In [7]:
nL, sL = openArray('refRelax')

In [10]:
def extract_spr(scoresList):
    
    outList = []
    #get total score and normalize to sequence length, final score needs associated sequence in different array place
    for x in scoresList:
        inner = []
        for y in range(len(x)):
            if y == 2:
                inner.append(x[y]['total_score']/len(x[1])) #score/res
            else:  
                inner.append(x[y])
        outList.append(inner)
        
    return outList

In [11]:
sL2 = extract_spr(sL)

In [13]:
cols =['name','seq','spr','time']
df = pd.DataFrame(sL2,columns=cols)

In [26]:
ax =sns.histplot(data=df,x='spr')

In [27]:
ax.figure.show()

In [19]:
df.head()

,name,seq,spr,time
0,0002182ba9e87d7b0d6b7d16b7001923_0001,SLDEILETLKELGVPPELLEEVKKVLEELERRGDPKREEVRRLLEE...,-2.643990,37.128287
1,000877380acb95e21726d9df052ec962_0001,DELEERVREALERGDPELLRKILRELVELLKKGDPEARKVLEILEE...,-2.550580,19.366846
2,0009c3faef123ed29e447cf47337d656_0001_1,SEDVEETAKELLERGTVEEFLKELKEELKERGDPELEKLLEKAEEL...,-2.515626,15.602136
3,000b40b97b3761973c100b7325f5bf41_0001_1,PEEEELKRLAKELGNPELAKLIELLLELLKQGDPDAEEALRLVKEL...,-2.489711,20.406318
4,001126bfe8b2f13d0bd52996ae0b77e8_0001_1,SDEELIEEIVEKLGLPPELREVARLIIELVKKGVPPEEVKELAKKL...,-2.770563,25.350820
